# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.233927088111                   -0.50    7.0         
  2   -7.250049741360       -1.79       -1.41    1.0   5.01ms
  3   -7.251284944803       -2.91       -2.05    2.0   6.07ms
  4   -7.251331571487       -4.33       -2.58    2.0   6.30ms
  5   -7.251333884790       -5.64       -2.71    1.0   5.14ms
  6   -7.251338492632       -5.34       -3.42    1.0   5.17ms
  7   -7.251338738842       -6.61       -3.74    2.0   6.29ms
  8   -7.251338790835       -7.28       -4.16    1.0   5.41ms
  9   -7.251338798178       -8.13       -4.95    1.0   5.45ms
 10   -7.251338798675       -9.30       -5.35    3.0   7.19ms
 11   -7.251338798703      -10.55       -5.89    2.0   6.81ms
 12   -7.251338798704      -12.19       -6.22    2.0   6.73ms
 13   -7.251338798705      -12.30       -6.97    1.0   5.71ms
 14   -7.251338798705      -13.50       -7.15    3.0   7.45ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.06557659838271114
[ Info: Arnoldi iteration step 2: normres = 0.585088414865014
[ Info: Arnoldi iteration step 3: normres = 0.8800436357354839
[ Info: Arnoldi iteration step 4: normres = 0.3358553282483788
[ Info: Arnoldi iteration step 5: normres = 0.254340717073938
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (2.34e-02, 3.75e-02, 1.56e-01, 1.94e-01, 2.76e-02)
[ Info: Arnoldi iteration step 6: normres = 0.4990060516803404
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (8.91e-03, 4.91e-02, 4.76e-01, 9.57e-02, 5.80e-02)
[ Info: Arnoldi iteration step 7: normres = 0.08820194381401761
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (4.29e-04, 8.58e-03, 3.84e-02, 2.60e-02, 6.67e-02)
[ Info: Arnoldi iteration step 8: normres = 0.10622925884494781
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.99e-05,